In [1]:
print('hello')

hello


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.optimize import minimize
import time
import talib as ta
from enum import Enum
import re

import pandas as pd
import numpy as np
from pathlib import Path
from scipy.optimize import minimize
import time
import talib as ta
from enum import Enum
import re
import os
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Tuple, Any
from datetime import datetime, timedelta

import sys
import matplotlib.pyplot as plt
from scipy.stats import zscore, kurtosis, skew, yeojohnson, boxcox
from scipy.stats import tukeylambda, mstats
from sklearn.preprocessing import RobustScaler
import zipfile
from io import BytesIO

In [4]:
class DataFrequency(Enum):
    """数据频率枚举"""
    MONTHLY = 'monthly'  # 月度数据
    DAILY = 'daily'      # 日度数据


def _generate_date_range(start_date: str, end_date: str, read_frequency: DataFrequency = DataFrequency.MONTHLY) -> List[str]:
    """
    生成日期范围列表
    
    参数:
    start_date: 起始日期
        - 月度格式: 'YYYY-MM' (如 '2020-01') 或 'YYYY-MM-DD' (自动转换为 'YYYY-MM')
        - 日度格式: 'YYYY-MM-DD' (如 '2020-01-01')
    end_date: 结束日期，格式同上
    frequency: 数据频率（月度或日度）
    
    返回:
    日期字符串列表
    """
    if read_frequency == DataFrequency.MONTHLY:
        # 兼容 'YYYY-MM' 和 'YYYY-MM-DD' 两种格式
        # 如果是 'YYYY-MM-DD' 格式，自动截取为 'YYYY-MM'
        new_start_date = start_date
        new_end_date = end_date
        if len(start_date) == 10:  # 'YYYY-MM-DD' 格式
            new_start_date = start_date[:7]
        if len(end_date) == 10:
            new_end_date = end_date[:7]
            
        start_dt = datetime.strptime(new_start_date, '%Y-%m')
        end_dt = datetime.strptime(new_end_date, '%Y-%m')
        
        date_list = []
        current_dt = start_dt
        while current_dt <= end_dt:
            date_list.append(current_dt.strftime('%Y-%m'))
            # 移动到下一个月
            if current_dt.month == 12:
                current_dt = current_dt.replace(year=current_dt.year + 1, month=1)
            else:
                current_dt = current_dt.replace(month=current_dt.month + 1)
        
        return date_list
    
    elif read_frequency == DataFrequency.DAILY:
        start_dt = datetime.strptime(start_date, '%Y-%m-%d')
        end_dt = datetime.strptime(end_date, '%Y-%m-%d')
        
        date_list = []
        current_dt = start_dt
        while current_dt <= end_dt:
            date_list.append(current_dt.strftime('%Y-%m-%d'))
            current_dt += timedelta(days=1)
        
        return date_list
    
    else:
        raise ValueError(f"不支持的数据频率: {frequency}")

In [ ]:
# start_date = '2025-01-01'
# end_date = '2025-11-01'
# read_frequency = DataFrequency.MONTHLY
# date_range_list = _generate_date_range(start_date=start_date, end_date=end_date, read_frequency=read_frequency)
# date_range_list

['2025-01',
 '2025-02',
 '2025-03',
 '2025-04',
 '2025-05',
 '2025-06',
 '2025-07',
 '2025-08',
 '2025-09',
 '2025-10',
 '2025-11']

处理多空数据
/Users/aming/data/ETHUSDT

takerlongshortRatio
topLongShortPositionRatio
topLongShortAccountRatio

大户的多头和空头总持仓量占比，大户指保证金余额排名前20%的用户。 
多仓持仓量比例 = 大户多仓持仓量 / 大户总持仓量 
空仓持仓量比例 = 大户空仓持仓量 / 大户总持仓量 
多空持仓量比值 = 多仓持仓量比例 / 空仓持仓量比例

topLongShortPositionRatio

https://developers.binance.com/docs/zh-CN/derivatives/usds-margined-futures/market-data/rest-api/Top-Trader-Long-Short-Ratio

{ 
         "symbol":"BTCUSDT",
	      "longShortRatio":"1.4342",// 大户多空持仓量比值
	      "longAccount": "0.5344", // 大户多仓持仓量比例
	      "shortAccount":"0.4238", // 大户空仓持仓量比例
	      "timestamp":"1583139600000"
}

名称	类型	是否必需	描述
symbol	STRING	YES	
period	ENUM	YES	"5m","15m","30m","1h","2h","4h","6h","12h","1d"
limit	LONG	NO	default 30, max 500
startTime	LONG	NO	
endTime	LONG	NO	


In [45]:
start_date = '2025-10-01'
end_date = '2025-11-01'
date_range_list = _generate_date_range(start_date=start_date, end_date=end_date, read_frequency=DataFrequency.MONTHLY)
dir = '/Users/aming/data/ETHUSDT'
path = 'topLongShortPositionRatio'
df_list = []

for date_str in date_range_list:
    df = pd.read_csv(f'{dir}/{path}/{path}_{date_str}.csv')
    df_list.append(df)

df = pd.concat(df_list)
df['open_time'] = pd.to_datetime(df['open_time'], unit='ns')
# df.sort_values(by='open_time', ascending=True, inplace=True)
df.set_index('open_time', inplace=True)
df.index = pd.to_datetime(df.index)
df.sort_index(inplace=True)
df
# df['buySellRatio'].plot()


,symbol,longAccount,longShortRatio,shortAccount
open_time,,,,
2025-10-01 00:00:00,ETHUSDT,0.7353,2.7775,0.2647
2025-10-01 00:00:00,ETHUSDT,0.7353,2.7775,0.2647
2025-10-01 00:00:00,ETHUSDT,0.7353,2.7775,0.2647
2025-10-01 00:00:00,ETHUSDT,0.7353,2.7775,0.2647
2025-10-01 00:00:00,ETHUSDT,0.7353,2.7775,0.2647
...,...,...,...,...
2025-11-30 23:50:00,ETHUSDT,0.7451,2.9237,0.2549
2025-11-30 23:50:00,ETHUSDT,0.7451,2.9237,0.2549
2025-11-30 23:50:00,ETHUSDT,0.7451,2.9237,0.2549


topLongShortAccountRatio

持仓大户的净持仓多头和空头账户数占比，大户指保证金余额排名前20%的用户。一个账户记一次。 多仓账户数比例 = 持多仓大户数 / 总持仓大户数 空仓账户数比例 = 持空仓大户数 / 总持仓大户数 多空账户数比值 = 多仓账户数比例 / 空仓账户数比例


https://developers.binance.com/docs/zh-CN/derivatives/usds-margined-futures/market-data/rest-api/Top-Long-Short-Account-Ratio

名称	类型	是否必需	描述
symbol	STRING	YES	
period	ENUM	YES	"5m","15m","30m","1h","2h","4h","6h","12h","1d"
limit	LONG	NO	default 30, max 500
startTime	LONG	NO	
endTime	LONG	NO	

{ 
         "symbol":"BTCUSDT",
	      "longShortRatio":"1.8105",// 大户多空账户数比值
	      "longAccount": "0.6442", // 大户多仓账户数比例
	      "shortAccount":"0.3558", // 大户空仓账户数比例
	      "timestamp":"1583139600000"
    }
    


In [46]:
start_date = '2025-10-01'
end_date = '2025-11-01'
date_range_list = _generate_date_range(start_date=start_date, end_date=end_date, read_frequency=DataFrequency.MONTHLY)
dir = '/Users/aming/data/ETHUSDT'
path = 'topLongShortAccountRatio'
df_list = []

for date_str in date_range_list:
    df = pd.read_csv(f'{dir}/{path}/{path}_{date_str}.csv')
    df_list.append(df)

df = pd.concat(df_list)
df['open_time'] = pd.to_datetime(df['open_time'], unit='ns')
# df.sort_values(by='open_time', ascending=True, inplace=True)
df.set_index('open_time', inplace=True)
df.index = pd.to_datetime(df.index)
df.sort_index(inplace=True)
df
# df['buySellRatio'].plot()


,symbol,longAccount,longShortRatio,shortAccount
open_time,,,,
2025-10-01 00:00:00,ETHUSDT,0.7126,2.4795,0.2874
2025-10-01 00:00:00,ETHUSDT,0.7126,2.4795,0.2874
2025-10-01 00:00:00,ETHUSDT,0.7126,2.4795,0.2874
2025-10-01 00:05:00,ETHUSDT,0.7122,2.4746,0.2878
2025-10-01 00:05:00,ETHUSDT,0.7122,2.4746,0.2878
...,...,...,...,...
2025-11-30 23:50:00,ETHUSDT,0.7028,2.3647,0.2972
2025-11-30 23:50:00,ETHUSDT,0.7028,2.3647,0.2972
2025-11-30 23:55:00,ETHUSDT,0.7037,2.3750,0.2963


{
    buySellRatio: "1.5586",
    buyVol: "387.3300", // 主动买入量
    sellVol: "248.5030", // 主动卖出量
    timestamp: "1585614900000",
  }

https://developers.binance.com/docs/zh-CN/derivatives/usds-margined-futures/market-data/rest-api/Taker-BuySell-Volume

takerlongshortRatio

In [15]:
start_date = '2025-10-01'
end_date = '2025-11-01'
date_range_list = _generate_date_range(start_date=start_date, end_date=end_date, read_frequency=DataFrequency.MONTHLY)
dir = '/Users/aming/data/ETHUSDT'
takerlongshortRatioPath = 'takerlongshortRatio'
df_list = []

for date_str in date_range_list:
    df = pd.read_csv(f'{dir}/{takerlongshortRatioPath}/{takerlongshortRatioPath}_{date_str}.csv')
    df_list.append(df)

df = pd.concat(df_list)
# df.sort_values(by='open_time', ascending=True, inplace=True)
df.set_index('open_time', inplace=True)
df.index = pd.to_datetime(df.index)
df.sort_index(inplace=True)
df.head()
# df['buySellRatio'].plot()


,buySellRatio,sellVol,buyVol
open_time,,,
2025-10-01 00:00:00,1.5031,4230.999,6359.645
2025-10-01 00:00:00,1.5031,4230.999,6359.645
2025-10-01 00:00:00,1.5031,4230.999,6359.645
2025-10-01 00:05:00,0.8976,3388.384,3041.311
2025-10-01 00:05:00,0.8976,3388.384,3041.311


In [ ]:
globalLongShortAccountRatio

https://developers.binance.com/docs/zh-CN/derivatives/usds-margined-futures/market-data/rest-api/Long-Short-Ratio

{ 
         "symbol":"BTCUSDT",
	      "longShortRatio":"0.1960", // 多空人数比值
	      "longAccount": "0.6622", // 多仓人数比例
	      "shortAccount":"0.3378", // 空仓人数比例
	      "timestamp":"1583139600000"
    
     }


In [49]:
start_date = '2025-10-01'
end_date = '2025-11-01'
date_range_list = _generate_date_range(start_date=start_date, end_date=end_date, read_frequency=DataFrequency.MONTHLY)
dir = '/Users/aming/data/ETHUSDT'
path = 'globalLongShortAccountRatio'
df_list = []

for date_str in date_range_list:
    df = pd.read_csv(f'{dir}/{path}/{path}_{date_str}.csv')
    df_list.append(df)

df = pd.concat(df_list)
# df.sort_values(by='open_time', ascending=True, inplace=True)
df.set_index('open_time', inplace=True)
df.index = pd.to_datetime(df.index)
df.sort_index(inplace=True)
df
# df.head()
# df['buySellRatio'].plot()


,symbol,longAccount,longShortRatio,shortAccount
open_time,,,,
2025-10-01 00:00:00,ETHUSDT,0.6786,2.1114,0.3214
2025-10-01 00:00:00,ETHUSDT,0.6786,2.1114,0.3214
2025-10-01 00:00:00,ETHUSDT,0.6786,2.1114,0.3214
2025-10-01 00:00:00,ETHUSDT,0.6786,2.1114,0.3214
2025-10-01 00:00:00,ETHUSDT,0.6786,2.1114,0.3214
...,...,...,...,...
2025-11-30 23:50:00,ETHUSDT,0.6650,1.9851,0.3350
2025-11-30 23:55:00,ETHUSDT,0.6655,1.9895,0.3345
2025-11-30 23:55:00,ETHUSDT,0.6655,1.9895,0.3345


杠杆数据

liquidations

In [24]:
start_date = '2025-10-01'
end_date = '2025-11-01'
read_frequency = DataFrequency.DAILY
date_range_list = _generate_date_range(start_date=start_date, end_date=end_date, read_frequency=read_frequency)
dir = '/Users/aming/data/ETHUSDT'
channel_path = 'liquidations'
symbol = 'ETHUSDT'
df_list = []

for date_str in date_range_list:
    df = pd.read_csv(f'{dir}/{channel_path}/binance-futures_{channel_path}_{date_str}_{symbol}.csv.gz')
    df_list.append(df)

df = pd.concat(df_list)
# df.sort_values(by='open_time', ascending=True, inplace=True)
df.rename(columns={'timestamp': 'open_time'}, inplace=True)
df['open_time'] = pd.to_datetime(df['open_time'], unit='us')
# df['funding_timestamp'] = pd.to_datetime(df['funding_timestamp'], unit='us')
df.set_index('open_time', inplace=True)
# df.index = pd.to_datetime(df.index)
df.sort_index(inplace=True)
df.drop(columns=['id', 'exchange', 'local_timestamp', 'symbol'], inplace=True)
df.head()
# df['buySellRatio'].plot()


,side,price,amount
open_time,,,
2025-10-01 00:00:17.767,buy,4161.21,3.244
2025-10-01 00:01:12.560,buy,4162.28,0.738
2025-10-01 00:01:39.582,buy,4164.18,0.060
2025-10-01 00:03:18.173,buy,4165.67,1.722
2025-10-01 00:05:02.358,buy,4166.02,3.837


derivative_ticker

In [ ]:
start_date = '2025-10-01'
end_date = '2025-11-01'
read_frequency = DataFrequency.DAILY
date_range_list = _generate_date_range(start_date=start_date, end_date=end_date, read_frequency=read_frequency)
dir = '/Users/aming/data/ETHUSDT'
channel_path = 'derivative_ticker'
symbol = 'ETHUSDT'
df_list = []

for date_str in date_range_list:
    df = pd.read_csv(f'{dir}/{channel_path}/binance-futures_{channel_path}_{date_str}_{symbol}.csv.gz')
    df_list.append(df)

df = pd.concat(df_list)
# df.sort_values(by='open_time', ascending=True, inplace=True)
df.rename(columns={'timestamp': 'open_time'}, inplace=True)
df['open_time'] = pd.to_datetime(df['open_time'], unit='us')
df['funding_timestamp'] = pd.to_datetime(df['funding_timestamp'], unit='us')
df.set_index('open_time', inplace=True)
# df.index = pd.to_datetime(df.index)
df.sort_index(inplace=True)
df.drop(columns=['exchange', 'local_timestamp', 'symbol'], inplace=True)
df.head()
# df['buySellRatio'].plot()


,funding_timestamp,funding_rate,predicted_funding_rate,open_interest,last_price,index_price,mark_price
open_time,,,,,,,


处理openInterest
{
	"openInterest": "10659.509", // 未平仓合约数量
	"symbol": "BTCUSDT",	// 交易对
	"time": 1589437530011   // 撮合引擎时间
}
https://developers.binance.com/docs/zh-CN/derivatives/usds-margined-futures/market-data/rest-api/Open-Interest

In [34]:
start_date = '2025-10-01'
end_date = '2025-11-01'
read_frequency = DataFrequency.MONTHLY
date_range_list = _generate_date_range(start_date=start_date, end_date=end_date, read_frequency=read_frequency)
dir = '/Users/aming/data/ETHUSDT'
channel_path = 'openInterest'
symbol = 'ETHUSDT'
df_list = []

for date_str in date_range_list:
    df = pd.read_csv(f'{dir}/{channel_path}/{channel_path}_{date_str}.csv')
    df_list.append(df)

df = pd.concat(df_list)
df.set_index('open_time', inplace=True)
# df.index = pd.to_datetime(df.index)
df.sort_index(inplace=True)
# df.drop(columns=['exchange', 'local_timestamp', 'symbol'], inplace=True)
df.head()
# df['buySellRatio'].plot()


,symbol,openInterest
open_time,,
2025-10-01 00:00:00.114,ETHUSDT,1827785.572
2025-10-01 00:00:05.408,ETHUSDT,1827799.596
2025-10-01 00:00:12.566,ETHUSDT,1827989.568
2025-10-01 00:00:20.574,ETHUSDT,1827917.962
2025-10-01 00:00:26.319,ETHUSDT,1827886.995
